<a href="https://colab.research.google.com/github/federicasibilla/GenAI_ecology/blob/main/Data_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Working on drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Directory **AI4Ecology** will be created at the root directory of drive to keep results.

In [3]:
google_drive_dir = "/content/gdrive/My Drive/AI4Ecology/"
!mkdir "$google_drive_dir"

# Imports

In [4]:
from time import time

import os
import glob
import shutil

import numpy as np
import itertools as it
import pandas as pd

# Pre-process data

## Notebook Output

This notebook saves a file (*parquet*) which contains a dataframe with right format to be used by following notebooks. Pre-processing is dependent of the initial format of the DB containing patches. Here you can see the example for the article DB.



# Processing

Read data from excel file. Then create a dataframe (X) where each column is a plant species and each row is a patch. Each cell in a row indicates if the patch contains (1) or not (0) a given species.

All other variables are removed from the dataframe. Dataframe X will be a table with 1 and 0 for each of the cells.

For simplicity only 8 plant species (*n_vars*) are kept in data, more abundant species. Relations between them will be studied.

**Important**: Variable *n_vars* is defined in other notebooks so it needs to be consistent between them.

In [5]:
# set the number of plant species or other similar variables to consider
n_vars=8
# set boolean to include soil type (whether gypsum or not) to be considered as another variable treated as a plant species
incl_soil = True
# (this could be generalized to location i.e. ravine/not, north_face/not, etc... alongside the present of a plant

# Download excel DB
!wget -q https://raw.githubusercontent.com/jegarcian/AI4Ecology/main/Data.xlsx -O 'Data.xlsx'

# Read excel file
df_no_trunc = pd.read_excel('Data.xlsx',sheet_name=2).iloc[:,:6]

Example of the DB used for this analysis before any processing

In [6]:
df_no_trunc.head()

,Localidad,Suelo,plotnumber,patchID,species,Nind
0,Barranc Fort,Matriz,plot1,P_1,Fumana_thymifolia,2
1,Barranc Fort,Matriz,plot1,P_1,Sedum_sediforme,1
2,Barranc Fort,Matriz,plot1,P_2,Cistus_clusii,1
3,Barranc Fort,Matriz,plot1,P_2,Fumana_thymifolia,1
4,Barranc Fort,Matriz,plot1,P_2,Helianthemum_violaceum,1


Do the data processing, **this will be dependent on the format of the initial patch DB**.

In [8]:
# from the column 'Suelo' define a column indicating wether the soil type is gypsum or not
df = df_no_trunc.rename(columns={'Suelo': 'Gypsum'})
tmp_condition = df['Gypsum'].isin(['Yeso'])
df.loc[tmp_condition, 'Gypsum'] = 1
df.loc[~tmp_condition, 'Gypsum'] = 0

#pivot 1D record into 2D table
df=df.pivot_table(index=['Localidad','plotnumber','patchID','Gypsum'],columns=['species'],values='Nind',fill_value=0)

#count whether a species occurs at all in a patch, instead of how many specimens appear per patch
df=df.clip(0,1)

#sort the columns according to abundance of plant species
if incl_soil == False:
    offset = 0
if incl_soil == True:
    offset = 1

df=df.reindex(df.sum(axis='rows').sort_values(ascending=False).index.tolist(),axis='columns').iloc[:,:n_vars-offset]

n_original_patches = df.shape[0]

#keep only patches with at least two distinct species
df=df[df.sum(axis='columns')>1]

# if desired,set Gypsum as a column in the same way as plant species are
if incl_soil == True:
    df=df.reset_index(drop=False)
    df=df.set_index(['Localidad','plotnumber','patchID'])

#list variables, i.e. plant species under consideration and soil type if selected
vars_range = range(n_vars)
var_names = df.columns.tolist()

# remove the information of patch ID and location
X=df.iloc[:,1-offset:n_vars-offset+1].reset_index(drop=True)
X

species,Gypsum,Fumana_thymifolia,Stipa_tenacissima,Helianthemum_syriacum,Helianthemum_squamatum,Teucrium_libanitis,Brachypodium_retusum,Fumana_ericoides
0,0,1,1,0,0,0,0,0
1,0,1,1,0,0,0,0,0
2,1,0,0,1,1,0,1,0
3,1,0,0,1,1,0,0,0
4,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...
2576,0,0,0,1,0,0,0,1
2577,1,0,0,0,1,1,0,0
2578,1,0,0,0,1,1,0,1
2579,1,0,0,0,0,1,0,1


Save dataframe as a *parquet* file.

In [9]:
dir_name = google_drive_dir+'saved_reals'
if not os.path.exists(dir_name):
        os.mkdir(dir_name)

file_name =os.path.join(dir_name,'X__n_vars='+str(n_vars)+'__incl_soil='+str(incl_soil)+'.parquet.gzip')
X.to_parquet(file_name,compression='gzip')

# Summary of Outputs produced

Output created at this point:

    saved_reals : Real data patches (real data)

Directory includes *parquet* file including DB in right format for later processing.

In [10]:
!ls -ltr "$google_drive_dir"

total 4
drwx------ 2 root root 4096 Jan 17 08:04 saved_reals
